In [ ]:
import glob

import pandas as pd

# show more columns
pd.options.display.max_columns = 500

# show the large integer as a regular number
pd.options.display.float_format = '{:.0f}'.format

In [ ]:
file_list = glob.glob("../data/*.parquet")
dfs = []

for file in file_list:
    df = pd.read_parquet(path=file)
    df['file'] = file
    dfs.append(df)

vehicles_data = pd.concat(dfs)

In [ ]:
logger_operations = pd.read_csv(filepath_or_buffer="../data/tires_vehicle_logger_operations.csv", sep=",")

## Task No. 0: Exploration

### Tires Vehicle Logger Operations

In [ ]:
display(logger_operations.info())
display(logger_operations.head(5))
display(logger_operations.tail(5))

In [ ]:
print(logger_operations.vehicle_licence_plate.nunique())
print(logger_operations.tireid.nunique())
print(logger_operations.loggerno.nunique())

In [ ]:
##8750061 8750076
##CU33706 DB30900
operational_data = logger_operations.loc[logger_operations['loggerno'].isin([8750061, 8750076]), :]
#operational_data = logger_operations.loc[logger_operations['vehicle_licence_plate'].isin(['CU33706', 'DB30900']), :]
operational_data.groupby(by=['loggerno', 'vehicle_licence_plate', 'tireid', 'wheelpos']).count()

### Vehicles data

In [ ]:
display(vehicles_data.info())
display(vehicles_data.head(5))
display(vehicles_data.tail(5))

In [ ]:
vehicles_data.describe()

In [ ]:
print(vehicles_data.loggerno.unique())

## Task No. 1: Understandable table and views

### Splitting the file column to get the logger ID.

In [ ]:
vehicles_data[['vin', 'loggerno', 'created_at', 'suffix']] = vehicles_data['file'].str.split('_', expand=True)
vehicles_data.drop(columns=['file', 'suffix'], inplace=True)
vehicles_data['vin'] = vehicles_data['vin'].str.replace(pat='^\./data/', repl='', regex=True)
vehicles_data['loggerno'] = vehicles_data['loggerno'].str.lstrip('0').astype(int)
vehicles_data['created_at'] = pd.to_datetime(vehicles_data['created_at'], format='%Y%m')

# drop unnecesary columns
if vehicles_data['ts'].equals(vehicles_data['ts_sec']):
    vehicles_data.drop(columns=['ts_sec'], inplace=True)
vehicles_data.drop(columns=['gps_long', 'gps_lat', 'alt', 'steeringwheel_angle', 'highres', 'vin'], inplace=True)

vehicles_data.head(3)


### Unpivoting the vehicle data

In [ ]:
def translate_wheel_position(wheel_position):
    translated_wheel_position = ''
    if wheel_position[0] == 'f':
        translated_wheel_position = '1' + wheel_position[1]
    else:
        translated_wheel_position = '2' + wheel_position[1]
    return translated_wheel_position.upper()

In [ ]:
up_vehicle_data = vehicles_data.melt(
    id_vars=['loggerno', 'created_at', 'temp_outside', 'tachometer_km', 'speed', 'ts', 'ts_int'], 
    var_name='variable', 
    value_name='value')

up_vehicle_data[['parameter', 'wheelpos']] = up_vehicle_data['variable'].str.split('_', expand=True)
up_vehicle_data.drop('variable', axis=1, inplace=True)

up_vehicle_data['wheelpos'] = up_vehicle_data['wheelpos'].apply(translate_wheel_position)

# add the value column to the end of the DataFrame
value_column = up_vehicle_data.pop('value')
up_vehicle_data = up_vehicle_data.assign(value=value_column)

up_vehicle_data.head(20)

In [ ]:
# (13544760, 10) -> (13513272, 10)
up_vehicle_data.drop_duplicates().to_parquet('../data/up_vehicle_data.parquet')

In [ ]:
up_vehicle_data = pd.read_parquet(path='../data/up_vehicle_data.parquet')

In [ ]:
selected_columns = [
    'loggerno',
    'tireid',
    'created_at',
    'temp_outside',
    'tachometer_km',
    'speed',
    'ts',
    'ts_int',
    'parameter',
    'wheelpos',
    'value'
]

df_task01 = pd.merge(left=up_vehicle_data, right=operational_data, how='inner', on=['loggerno', 'wheelpos'])

df_task01.loc[:, selected_columns].to_parquet(path='../data/task01.parquet')